In [22]:
import numpy as np
import pandas as pd
import os
import math
from scipy.signal import butter, lfilter, freqz
import matplotlib.pyplot as plt

In [52]:
paths = []

for i in range(1, 16):
    root, file = os.path.split("../Dataset/SA01/F01_SE06_R01.csv")
    if i < 10:
        file = file[:2] + str(i) + file[3:]
        tmp = os.path.join(root, file)
        paths.append(tmp)
        if i > 4:
            for j in range(2, 6):
                file_new = file[:-5] + str(j) + file[-4:]
                tmp = os.path.join(root, file_new)
                paths.append(tmp)
    if i >= 10:
        file = file[:1] + str(i) + file[3:]
        for j in range(1, 6):
            file = file[:-5] + str(j) + file[-4:]
            tmp = os.path.join(root, file)
            paths.append(tmp)

# paths[4:14] = ['../Dataset/SA01/D05_SA01_R01_1.csv',
#                 '../Dataset/SA01/D05_SA01_R01_2.csv',
#                 '../Dataset/SA01/D05_SA01_R02_1.csv',
#                 '../Dataset/SA01/D05_SA01_R02_2.csv',
#                 '../Dataset/SA01/D05_SA01_R03_1.csv',
#                 '../Dataset/SA01/D05_SA01_R03_2.csv',
#                 '../Dataset/SA01/D05_SA01_R04_1.csv',
#                 '../Dataset/SA01/D05_SA01_R04_2.csv',
#                 '../Dataset/SA01/D05_SA01_R05_1.csv',
#                 '../Dataset/SA01/D05_SA01_R05_2.csv',
#                 '../Dataset/SA01/D06_SA01_R01_1.csv',
#                 '../Dataset/SA01/D06_SA01_R01_2.csv',
#                 '../Dataset/SA01/D06_SA01_R02_1.csv',
#                 '../Dataset/SA01/D06_SA01_R02_2.csv',
#                 '../Dataset/SA01/D06_SA01_R03_1.csv',
#                 '../Dataset/SA01/D06_SA01_R03_2.csv',
#                 '../Dataset/SA01/D06_SA01_R04_1.csv',
#                 '../Dataset/SA01/D06_SA01_R04_2.csv',
#                 '../Dataset/SA01/D06_SA01_R05_1.csv',
#                 '../Dataset/SA01/D06_SA01_R05_2.csv']

# paths[-15:-10] = ['../Dataset/SA01/D17_SA01_R01_1.csv',
#                  '../Dataset/SA01/D17_SA01_R01_2.csv',
#                  '../Dataset/SA01/D17_SA01_R02_1.csv',
#                  '../Dataset/SA01/D17_SA01_R02_2.csv',
#                  '../Dataset/SA01/D17_SA01_R03_1.csv',
#                  '../Dataset/SA01/D17_SA01_R03_2.csv',
#                  '../Dataset/SA01/D17_SA01_R04_1.csv',
#                  '../Dataset/SA01/D17_SA01_R04_2.csv',
#                  '../Dataset/SA01/D17_SA01_R05_1.csv',
#                 '../Dataset/SA01/D17_SA01_R05_2.csv']

SE06_F_paths = []

for path in paths:
    path = path.replace('SA01', 'SE06')
    SE06_F_paths.append(path)

SE06_F_paths

['../Dataset/SE06\\F01_SE06_R01.csv',
 '../Dataset/SE06\\F02_SE06_R01.csv',
 '../Dataset/SE06\\F03_SE06_R01.csv',
 '../Dataset/SE06\\F04_SE06_R01.csv',
 '../Dataset/SE06\\F05_SE06_R01.csv',
 '../Dataset/SE06\\F05_SE06_R02.csv',
 '../Dataset/SE06\\F05_SE06_R03.csv',
 '../Dataset/SE06\\F05_SE06_R04.csv',
 '../Dataset/SE06\\F05_SE06_R05.csv',
 '../Dataset/SE06\\F06_SE06_R01.csv',
 '../Dataset/SE06\\F06_SE06_R02.csv',
 '../Dataset/SE06\\F06_SE06_R03.csv',
 '../Dataset/SE06\\F06_SE06_R04.csv',
 '../Dataset/SE06\\F06_SE06_R05.csv',
 '../Dataset/SE06\\F07_SE06_R01.csv',
 '../Dataset/SE06\\F07_SE06_R02.csv',
 '../Dataset/SE06\\F07_SE06_R03.csv',
 '../Dataset/SE06\\F07_SE06_R04.csv',
 '../Dataset/SE06\\F07_SE06_R05.csv',
 '../Dataset/SE06\\F08_SE06_R01.csv',
 '../Dataset/SE06\\F08_SE06_R02.csv',
 '../Dataset/SE06\\F08_SE06_R03.csv',
 '../Dataset/SE06\\F08_SE06_R04.csv',
 '../Dataset/SE06\\F08_SE06_R05.csv',
 '../Dataset/SE06\\F09_SE06_R01.csv',
 '../Dataset/SE06\\F09_SE06_R02.csv',
 '../Dataset

In [53]:
def butter_lowpass(cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=4):
    b, a = butter_lowpass(cutoff, fs, order=order)
    x = np.trunc(np.array(data, dtype=np.float64)).astype(np.int64)
    y = lfilter(b, a, x, axis=-1, zi=None)
    return y

# Setting standard filter requirements.
order = 4
fs = 200
cutoff = 5

b, a = butter_lowpass(cutoff, fs, order)

# Creating the data for filteration
T = 200         # value taken in seconds
n = int(T * fs) # indicates total samples
t = np.linspace(0, T, n, endpoint=False)

In [78]:
list = []
LPF_list = []
final_SA22_F = pd.DataFrame(list)

for path in SA22_F_paths:
    try:
        data = pd.read_csv(path, header=None)
        data = data.iloc[:, :3]
        rows, cols = data.shape

        sliced_df = data[0 : 160]
    
        # LPF 구하기

        col_1_lpf = butter_lowpass_filter(sliced_df[0].to_numpy(), cutoff, fs, order).astype('float64')
        col_2_lpf = butter_lowpass_filter(sliced_df[1].to_numpy(), cutoff, fs, order).astype('float64')
        col_3_lpf = butter_lowpass_filter(sliced_df[2].to_numpy(), cutoff, fs, order).astype('float64')
    
    
        col_1_std = np.sqrt(np.absolute(np.mean(sliced_df[0]) - sliced_df[0])**2 / len(sliced_df[0]))
        col_2_std = np.sqrt(np.absolute(np.mean(sliced_df[1]) - sliced_df[1])**2 / len(sliced_df[1]))
        col_3_std = np.sqrt(np.absolute(np.mean(sliced_df[2]) - sliced_df[2])**2 / len(sliced_df[2]))
    
        # 가속도 구하기
        # 표준편차 구하기

        col_1 = col_1_std*col_1_std
        col_2 = col_2_std*col_2_std
        col_3 = col_3_std*col_3_std
    
        C_nine = np.sqrt(col_1 + col_2 + col_3)
        list_C_nine = pd.DataFrame(C_nine).transpose().round(3)
        final_SA22_F = final_SA22_F.append(list_C_nine)
        
    except:
        pass

pd.set_option('display.max_row', 500)
pd.set_option('display.max_columns', 100)

row, col = final_SA22_F.shape
target_col = row * [1]
final_SA22_F.loc[:,'target'] = target_col

final_SA22_F

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,target
0,1.339,1.795,1.795,1.947,2.137,2.199,2.357,1.995,2.861,1.870,1.740,1.769,1.649,1.535,1.254,1.284,1.485,1.506,1.574,1.429,1.415,1.905,2.428,2.653,3.210,3.615,4.026,4.278,4.411,4.276,4.358,4.068,3.636,3.204,2.473,2.196,1.875,1.922,2.202,2.768,3.137,3.627,3.905,4.175,4.314,4.659,4.617,4.687,4.453,4.329,...,4.351,4.154,3.689,3.615,3.010,2.345,1.967,1.418,1.003,0.735,1.248,1.011,1.418,1.739,1.842,1.976,1.787,1.670,1.584,1.434,1.381,1.548,1.391,1.427,1.071,1.142,1.074,1.059,0.803,0.790,0.692,0.827,0.580,0.900,1.112,1.362,1.525,1.534,1.648,1.788,1.686,1.719,1.782,1.964,1.608,1.622,1.675,1.919,1.904,1
0,1.789,1.513,1.541,1.212,0.956,1.090,0.563,0.687,0.641,0.612,0.825,0.653,0.941,0.895,0.777,1.057,1.379,1.287,1.705,1.994,2.203,2.222,2.284,2.268,2.495,2.549,2.516,2.470,2.635,2.459,2.604,2.981,3.238,3.486,3.760,3.770,4.034,3.471,3.603,3.105,2.949,2.497,2.405,1.560,1.914,2.099,2.177,2.512,2.507,2.596,...,0.286,0.500,0.381,0.560,0.450,0.304,0.597,0.819,0.928,1.301,1.848,2.111,2.673,2.682,3.062,3.095,3.374,3.610,3.471,3.541,3.659,3.823,4.116,4.614,4.829,4.650,4.755,4.559,4.710,4.765,4.763,4.527,4.146,3.940,3.532,3.021,2.647,2.127,1.893,1.732,1.935,1.763,1.549,1.882,1.752,2.126,2.354,2.481,2.397,1
0,0.247,0.515,0.212,0.331,0.311,0.200,0.091,0.385,0.383,0.216,0.343,0.371,0.160,0.203,0.171,0.123,0.418,0.275,0.372,0.250,0.436,0.487,0.399,0.250,0.275,0.378,0.286,0.389,0.146,0.326,0.409,0.409,0.408,0.340,0.195,0.274,0.376,0.209,0.384,0.181,0.211,0.195,0.424,0.538,0.150,0.266,0.707,0.483,0.437,0.559,...,0.275,0.437,0.103,0.499,0.091,0.150,0.249,0.321,0.209,0.294,0.537,0.464,0.520,0.257,0.574,0.523,0.449,0.384,0.158,0.107,0.447,0.190,0.176,0.399,0.320,0.527,0.128,0.311,0.294,0.273,0.324,0.383,0.209,0.420,0.412,0.503,0.372,0.055,0.381,0.219,0.463,0.453,0.566,0.506,0.662,0.736,0.559,0.843,0.628,1
0,0.532,0.653,0.651,0.495,0.131,0.586,1.218,0.992,0.785,0.574,0.139,1.411,0.131,0.265,0.276,0.336,0.280,0.335,0.541,0.364,0.292,0.260,0.322,0.362,0.480,0.437,0.390,0.280,0.509,0.511,0.432,0.614,0.367,0.472,0.325,0.417,0.106,0.384,0.216,0.078,0.255,0.176,0.154,0.181,0.328,0.412,0.630,0.499,0.603,0.832,...,0.243,0.252,0.280,0.210,0.200,0.232,0.369,0.078,0.510,0.131,0.338,0.373,0.384,0.259,0.521,0.640,0.441,0.337,0.403,0.335,0.226,0.453,0.448,0.395,0.580,0.417,0.296,0.498,0.346,0.735,0.358,0.327,0.460,0.652,0.676,0.410,0.686,0.720,0.725,0.514,0.525,0.742,0.652,0.743,0.752,0.969,0.848,1.290,1.502,1
0,1.482,1.410,1.213,1.237,0.953,1.486,1.091,1.115,0.845,0.944,1.218,1.336,1.054,1.290,1.210,1.097,1.134,1.093,1.485,1.425,1.563,1.508,1.841,1.893,1.804,1.886,2.404,2.457,2.733,2.842,3.239,3.710,3.576,3.797,4.166,4.123,3.956,3.620,3.303,3.182,2.549,2.227,1.821,1.515,1.069,0.816,0.995,1.199,1.810,2.024,...,1.619,1.756,1.919,1.781,1.934,2.057,1.978,2.141,2.298,2.455,2.563,2.669,2.499,2.821,2.633,2.545,2.660,2.447,2.659,2.666,2.567,2.607,2.659,2.576,2.284,2.123,2.169,2.443,2.591,2.559,3.078,3.521,3.835,4.176,4.192,4.307,4.236,4.235,4.275,4.195,4.268,4.471,4.445,4.604,4.619,4.464,4.274,3.764,3.312,1
0,0.767,0.715,0.927,1.109,1.187,1.542,1.271,0.908,1.212,0.935,1.309,1.368,0.918,0.560,1.150,1.053,0.970,0.875,0.532,0.779,0.753,0.710,0.653,0.897,0.676,0.770,0.694,0.754,0.544,0.597,0.520,0.590,0.510,0.882,0.377,0.736,0.686,0.678,0.898,0.363,0.576,0.065,0.341,0.065,0.198,0.190,0.299,0.412,0.402,0.543,...,1.592,2.133,2.305,2.491,3.025,3.147,3.187,3.216,2.882,2.775,2.397,2.231,1.532,1.376,1.137,1.016,1.196,1.240,1.597,1.594,1.992,2.022,2.328,2.258,1.978,1.880,1.842,1.318,1.313,0.843,0.570,0.550,0.951,1.020,1.287,1.398,1.825,1.844,1.805,2.139,1.775,1.968,1.730,1.621,1.070,1.454,1.045,1.058,1.234,1
0,0.528,0.795,0.857,0.781,0.588,1.148,1.

In [79]:
A01 = final_SA01_F

In [80]:
A02 = final_SA02_F

In [81]:
A03 = final_SA03_F

In [82]:
A04 = final_SA04_F

In [83]:
A05 = final_SA05_F

In [84]:
A06 = final_SA06_F

In [85]:
A07 = final_SA07_F

In [86]:
A08 = final_SA08_F

In [87]:
A09 = final_SA09_F

In [88]:
A10 = final_SA10_F

In [89]:
A11 = final_SA11_F

In [90]:
A12 = final_SA12_F

In [91]:
A13 = final_SA13_F

In [92]:
A14 = final_SA14_F

In [93]:
A15 = final_SA15_F

In [94]:
A16 = final_SA16_F

In [95]:
A17 = final_SA17_F

In [96]:
A18 = final_SA18_F

In [97]:
A19 = final_SA19_F

In [98]:
A20 = final_SA20_F

In [99]:
A21 = final_SA21_F

In [100]:
A22 = final_SA22_F

In [101]:
A23 = final_SA23_F

In [102]:
E06 = final_SE06_F

In [103]:
FALL_all = pd.concat([A01, A02, A03, A04, A05, A06, A07, A08, A09, A10, A11, A12, A13, A14, A15, A16, A17, A18, A19, A20, A21, A22, A23, E06])

In [104]:
FALL_all.to_csv('[C9]LPF_FALL.csv',index=False)

In [106]:
df_1 = pd.read_csv('[C9]LPF_ADL.csv')

In [107]:
df_2 = pd.read_csv('[C9]LPF_FALL.csv')

In [109]:
df = pd.concat([df_1,df_2])

In [111]:
df.to_csv('[C9]_LPF.csv', index=False)